In [13]:
import clingo
import telingo
from clingo import ast

In [14]:
inst1 = """
#program always.

node(1).
node(2).
node(3).

edge((1,2)).
edge((1,3)).
edge((2,1)).
edge((2,3)).
edge((3,1)).
edge((3,2)).

edge_weight((1,2), 10).
edge_weight((2,3), 10).
edge_weight((3,1), 10).
edge_weight((2,1), 100).
edge_weight((3,2), 100).
edge_weight((1,3), 100).

"""

In [15]:
reasoning = """

#program always.

walk(E) :-
  edge(E),
  not else(E).

else(E1) :-
  edge(E1), edge(E2), E1 != E2,
  walk(E2).

cost(C+W) :- 'cost(C), walk(E), edge_weight(E,W).

#show walk/1.
#show cost/1.

#program initial.

else((U,V)) :-
  edge((U,V)), U != 1.

:- edge((U,V)), edge((V,U)), not &tel { >? (walk((U,V)) | walk((V,U))) }.

cost(W) :- walk(E), edge_weight(E,W).

#program dynamic.

else((W,X)) :-
  edge((W,X)),
  'walk((U,V)), V != W.

#program final.

else((W,X)) :-
  edge((W,X)), X != 1.

"""

In [16]:
program = (inst1, reasoning)

In [17]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0

with ast.ProgramBuilder(ctl) as b:
    future_sigs, program_parts = telingo._tf.transform(program, b.add)

telingo.imain(ctl, future_sigs, program_parts, on_model=print, imin=2, imax=5, istop="UNKNOWN")

walk((1,3),0) walk((3,2),1) walk((2,1),2) cost(100,0) cost(200,1) cost(300,2) 2
walk((1,2),0) walk((2,3),1) walk((3,1),2) cost(10,0) cost(20,1) cost(30,2) 2
walk((1,3),0) walk((3,2),1) walk((2,1),2) cost(100,0) cost(200,1) cost(300,2) 3
walk((1,2),0) walk((2,3),1) walk((3,1),2) cost(10,0) cost(20,1) cost(30,2) 3
walk((1,2),0) walk((2,3),1) walk((3,1),2) walk((1,2),3) walk((2,1),4) cost(10,0) cost(20,1) cost(30,2) cost(40,3) cost(140,4) 4
walk((1,2),0) walk((2,3),1) walk((3,1),2) walk((1,3),3) walk((3,1),4) cost(10,0) cost(20,1) cost(30,2) cost(130,3) cost(140,4) 4
walk((1,2),0) walk((2,3),1) walk((3,2),2) walk((2,3),3) walk((3,1),4) cost(10,0) cost(20,1) cost(120,2) cost(130,3) cost(140,4) 4
walk((1,3),0) walk((3,2),1) walk((2,1),2) walk((1,2),3) walk((2,1),4) cost(100,0) cost(200,1) cost(300,2) cost(310,3) cost(410,4) 4
walk((1,3),0) walk((3,2),1) walk((2,1),2) walk((1,3),3) walk((3,1),4) cost(100,0) cost(200,1) cost(300,2) cost(400,3) cost(410,4) 4
walk((1,2),0) walk((2,1),1) walk((1